In [13]:
import pandas as pd


from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired


/home/sebastien/.pyenv/versions/3.10.6/envs/NLP-Systematic-review/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/sebastien/.pyenv/versions/3.10.6/envs/NLP-Systematic-review/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/se

In [144]:
from google.cloud import bigquery

# Variable
GCP_PROJECT_SEBT84='useful-proposal-392710'
GCP_REGION='europe-west1'

# BigQuery
BQ_REGION='EU'
BQ_DATASET='enhancing_systemic_reviews_nlp'
TABLE = 'pubmed'

# Cloud Storage
BUCKET_NAME='enhancing_systemic_reviews_nlp'

In [14]:
df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/data.csv')
topic_model = BERTopic()

In [136]:
#size = 2000
df_size = df.shape[0]
frac = 0.05
sample_size = round(df_size*frac)
display(f'sample size: {sample_size}')

df_small = df.iloc[0:sample_size]

df_train = df_small['abstract_text']
df_train.shape

'sample size: 9533'

(9533,)

In [137]:
%%time

topics, probs = topic_model.fit_transform(df_train)

CPU times: user 1h 22min 44s, sys: 9min 36s, total: 1h 32min 20s
Wall time: 16min 43s


In [138]:
print(len(topics))
df_train.shape
topic_model.get_document_info(df_train)


9533


,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,We conducted this study to assess the clinical...,188,188_raynaud_iloprost_phenomenon_digital,"[raynaud, iloprost, phenomenon, digital, skin,...",[To compare the long term effects of short ter...,raynaud - iloprost - phenomenon - digital - sk...,1.000000,True
1,To determine whether prophylactic treatment wi...,20,20_heparin_thrombosis_enoxaparin_venous,"[heparin, thrombosis, enoxaparin, venous, dvt,...",[The risk of venous thromboembolism in patient...,heparin - thrombosis - enoxaparin - venous - d...,0.650219,False
2,After the discovery of type C hepatitis virus ...,85,85_interferon_hepatitis_hcv_rna,"[interferon, hepatitis, hcv, rna, sustained, c...",[After the discovery of type C hepatitis virus...,interferon - hepatitis - hcv - rna - sustained...,1.000000,True
3,Since it is not clear whether testosterone or ...,5,5_bone_calcium_density_mineral,"[bone, calcium, density, mineral, spine, postm...",[To assess the effect of the antiestrogenic ag...,bone - calcium - density - mineral - spine - p...,0.944850,False
4,The aim was to study the pharmacokinetic param...,186,186_igfi_nitrogen_growth_gh,"[igfi, nitrogen, growth, gh, rhigfi, hormone, ...",[Earlier clinical studies have demonstrated im...,igfi - nitrogen - growth - gh - rhigfi - hormo...,1.000000,False
...,...,...,...,...,...,...,...,...
9528,To determine effect of prophylactic treatment ...,5,5_bone_calcium_density_mineral,"[bone, calcium, density, mineral, spine, postm...",[To assess the effect of the antiestrogenic ag...,bone - calcium - density - mineral - spine - p...,0.459389,False
9529,To evaluate the sedative properties of 3 dosag...,0,0_propofol_anesthesia_midazolam_isoflurane,"[propofol, anesthesia, midazolam, isoflurane, ...",[To determine if a total intravenous ( i.v. ) ...,propofol - anesthesia - midazolam - isoflurane...,1.000000,False
9530,Acute cough with purulent sputum is a common c...,64,64_theophylline_aminophylline_asthma_nedocromil,"[theophylline, aminophylline, asthma, nedocrom...",[To determine the efficacy of theophylline whe...,theophylline - aminophylline - asthma - nedocr...,0.716061,False
9531,The authors present the results of two paralle...,74,74_bladder_bcg_intravesical_superficial,"[bladder, bcg, intravesical, superficial, recu...",[We compared the efficacy and toxicity of long...,bladder - bcg - intravesical - superficial - r...,0.893414,False


In [139]:
df_with_topics = pd.concat([df_small['abstract_id'],topic_model.get_document_info(df_train)],axis=1)
df_with_topics # to be loaded into BQ

,abstract_id,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,1279170,We conducted this study to assess the clinical...,188,188_raynaud_iloprost_phenomenon_digital,"[raynaud, iloprost, phenomenon, digital, skin,...",[To compare the long term effects of short ter...,raynaud - iloprost - phenomenon - digital - sk...,1.000000,True
1,1281030,To determine whether prophylactic treatment wi...,20,20_heparin_thrombosis_enoxaparin_venous,"[heparin, thrombosis, enoxaparin, venous, dvt,...",[The risk of venous thromboembolism in patient...,heparin - thrombosis - enoxaparin - venous - d...,0.650219,False
2,1282364,After the discovery of type C hepatitis virus ...,85,85_interferon_hepatitis_hcv_rna,"[interferon, hepatitis, hcv, rna, sustained, c...",[After the discovery of type C hepatitis virus...,interferon - hepatitis - hcv - rna - sustained...,1.000000,True
3,1283117,Since it is not clear whether testosterone or ...,5,5_bone_calcium_density_mineral,"[bone, calcium, density, mineral, spine, postm...",[To assess the effect of the antiestrogenic ag...,bone - calcium - density - mineral - spine - p...,0.944850,False
4,1283730,The aim was to study the pharmacokinetic param...,186,186_igfi_nitrogen_growth_gh,"[igfi, nitrogen, growth, gh, rhigfi, hormone, ...",[Earlier clinical studies have demonstrated im...,igfi - nitrogen - growth - gh - rhigfi - hormo...,1.000000,False
...,...,...,...,...,...,...,...,...,...
9528,8790549,To determine effect of prophylactic treatment ...,5,5_bone_calcium_density_mineral,"[bone, calcium, density, mineral, spine, postm...",[To assess the effect of the antiestrogenic ag...,bone - calcium - density - mineral - spine - p...,0.459389,False
9529,8790552,To evaluate the sedative properties of 3 dosag...,0,0_propofol_anesthesia_midazolam_isoflurane,"[propofol, anesthesia, midazolam, isoflurane, ...",[To determine if a total intravenous ( i.v. ) ...,propofol - anesthesia - midazolam - isoflurane...,1.000000,False
9530,8790652,Acute cough with purulent sputum is a common c...,64,64_theophylline_aminophylline_asthma_nedocromil,"[theophylline, aminophylline, asthma, nedocrom...",[To determine the efficacy of theophylline whe...,theophylline - aminophylline - asthma - nedocr...,0.716061,False
9531,8791042,The authors present the results of two paralle...,74,74_bladder_bcg_intravesical_superficial,"[bladder, bcg, intravesical, superficial, recu...",[We compared the efficacy and toxicity of long...,bladder - bcg - intravesical - superficial - r...,0.893414,False


In [286]:
def load_data_bq(df: pd.DataFrame
                 , replace: bool):
    full_table_name = f"{GCP_PROJECT_SEBT84}.{BQ_DATASET}.topics_{TABLE}"

    client = bigquery.Client()

    write_mode = "WRITE_TRUNCATE" if replace else "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)

    job = client.load_table_from_dataframe(df, full_table_name, job_config=job_config)
    result = job.result()

    print(f"✅ Data saved to bigquery, with shape {df.shape}")

In [298]:
df = pd.read_csv('/home/sebastien/code/CuevasEli/NLP-Systematic-review/raw_data/df_with_topics.csv', delimiter="\t")#,engine='python')


In [297]:
df.shape

(15218, 1)

In [179]:
def get_data_from_bq(query):
    
    client = bigquery.Client(project=GCP_PROJECT_SEBT84)

    query_job = client.query(query)
    results = query_job.result()  # Waits for job to complete.
    df = results.to_dataframe()

    print(f"✅ Data loaded, with shape {df.shape}")

    return df

In [278]:
load_data_bq(df_with_topics , replace=True)

✅ Data saved to bigquery, with shape (9533, 9)


In [149]:
path = 'nlp_systematic_review/model'
topic_model.save(path, serialization="safetensors")

In [150]:
loaded_model = BERTopic.load(path)

In [152]:
loaded_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1832,-1_the_and_of_in,"[the, and, of, in, to, were, patients, with, w...",NaN
1,0,228,0_propofol_anesthesia_midazolam_isoflurane,"[propofol, anesthesia, midazolam, isoflurane, ...",NaN
2,1,222,1_pressure_blood_diastolic_hypertension,"[pressure, blood, diastolic, hypertension, hg,...",NaN
3,2,171,2_back_pain_strength_exercise,"[back, pain, strength, exercise, training, mot...",NaN
4,3,143,3_breast_tamoxifen_cancer_adjuvant,"[breast, tamoxifen, cancer, adjuvant, survival...",NaN
...,...,...,...,...,...
208,207,11,207_cortisol_anesthesia_catecholamine_postoper...,"[cortisol, anesthesia, catecholamine, postoper...",NaN
209,208,11,208_desipramine_stuttering_autism_autistic,"[desipramine, stuttering, autism, autistic, na...",NaN
210,209,11,209_cholesterol_advice_mmoll_guidelines,"[cholesterol, advice, mmoll, guidelines, serum...",NaN
211,210,11,210_pneumonia_clarithromycin_pneumoniae_commun...,"[pneumonia, clarithromycin, pneumoniae, commun...",NaN


In [155]:
query = "article about lung cancer patient care"

query_topics = topic_model.find_topics(query)

In [279]:
topic_list_str = str(query_topics[0]).replace('[','').replace(']','')

#f"{str(query_topics[0])}"
table = f"topics_{TABLE}"
query = f"""
        SELECT 
          *
        FROM {GCP_PROJECT_SEBT84}.{BQ_DATASET}.{table}
        WHERE Topic in ({topic_list_str})
        """

#query
articles = get_data_from_bq(query)

✅ Data loaded, with shape (189, 9)


In [271]:
#get topis
articles_slim = articles[['abstract_id','Topic','Probability']]
articles_slim['article_url'] = articles_slim['abstract_id'].apply(lambda x: 'https://pubmed.ncbi.nlm.nih.gov/'+str(x))

query = "cancer skin patient care"

query_topics = topic_model.find_topics(query)
topic_info = topic_model.get_topic_info()

topic_list =[]
for t in range(len(query_topics[0])):
    topic = topic_info[topic_info['Topic'] == query_topics[0][t]]
    
    topic_id = list(topic['Topic'])[0]
    topic_prob = query_topics[1][t]
    topic_name = list(topic['Name'])[0]
    topic_representation = list(topic['Representation'])[0]

    article = articles_slim[articles_slim['Topic'] == topic_id]
    
    article_count = article.shape[0]

    article_list = []
    for r in range(article_count):
        article_id = article.iloc[r][0]
        article_prob = article.iloc[r][2]
        article_url = article.iloc[r][3]
        article_list.append(dict({'article_id':str(article_id)
                              ,'article_prob':str(article_prob)
                              ,'article_url':article_url}))
    
    topic_dict = dict({"topic_id":str(topic_id)
                       ,"topic_prob":topic_prob
                       ,"topic_name":topic_name
                       ,"topic_representation":topic_representation
                       ,'article_count':article_count
                       ,'article_list':article_list
                      })

    topic_list.append(topic_dict)
    
dict({'return':topic_list})
    


{'return': [{'topic_id': '27',
   'topic_prob': 0.54234946,
   'topic_name': '27_skin_tretinoin_cream_vehicle',
   'topic_representation': ['skin',
    'tretinoin',
    'cream',
    'vehicle',
    'lesions',
    'topical',
    'dermatitis',
    'acne',
    'epidermal',
    'isotretinoin'],
   'article_count': 0,
   'article_list': []},
  {'topic_id': '71',
   'topic_prob': 0.3906642,
   'topic_name': '71_ulcers_healing_foot_ulcer',
   'topic_representation': ['ulcers',
    'healing',
    'foot',
    'ulcer',
    'shoes',
    'dressings',
    'wound',
    'dressing',
    'diabetic',
    'beds'],
   'article_count': 0,
   'article_list': []},
  {'topic_id': '39',
   'topic_prob': 0.36487514,
   'topic_name': '39_neck_radiotherapy_head_radiation',
   'topic_representation': ['neck',
    'radiotherapy',
    'head',
    'radiation',
    'gy',
    'survival',
    'carcinoma',
    'squamous',
    'tumor',
    'arm'],
   'article_count': 0,
   'article_list': []},
  {'topic_id': '112',
   'top

In [193]:
articles.head(2)
print(query_topics)
print(articles['Topic'].value_counts())
list(articles['Topic'].value_counts())

([8, 89, 193, 175, 42], [0.5207312, 0.42068022, 0.39886177, 0.3979378, 0.36991405])
Topic
8      102
42      47
89      30
175     15
193     13
Name: count, dtype: int64


[102, 47, 30, 15, 13]

In [280]:
#all articles + slim down the dataset
articles_slim = articles[['abstract_id','Topic','Probability']]
articles_slim['article_url'] = articles_slim['abstract_id'].apply(lambda x: 'https://pubmed.ncbi.nlm.nih.gov/'+str(x))

article = articles_slim[articles_slim['Topic'] == topic_id]

article_count = article.shape[0]

article_list = []
for r in range(article.shape[0]):
    article_id = article.iloc[r][0]
    article_prob = article.iloc[r][2]
    article_url = article.iloc[r][3]
    article_list.append(dict({'article_id':str(article_id)
                              ,'article_prob':str(article_prob)
                              ,'article_url':article_url}))

article_list

[{'article_id': '1903663',
  'article_prob': '0.6299437377475179',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/1903663'},
 {'article_id': '1997835',
  'article_prob': '0.9284397941035724',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/1997835'},
 {'article_id': '7972904',
  'article_prob': '0.7241424873668774',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/7972904'},
 {'article_id': '8083097',
  'article_prob': '0.7582263049476891',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/8083097'},
 {'article_id': '8430362',
  'article_prob': '0.9611773563287532',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/8430362'},
 {'article_id': '8514551',
  'article_prob': '0.6626673591412224',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/8514551'},
 {'article_id': '8672151',
  'article_prob': '0.7378356532559841',
  'article_url': 'https://pubmed.ncbi.nlm.nih.gov/8672151'},
 {'article_id': '8683634',
  'article_prob': '0.8960166782800036',
  'article_url': 'https://pubmed.ncbi

In [285]:
#article = articles_slim[articles_slim['Topic'] == 5]
articles_slim[articles_slim['Topic'] == 71].sort_values('Probability',ascending=False)

,abstract_id,Topic,Probability,article_url
168,8673309,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/8673309
166,8304833,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/8304833
159,1463945,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/1463945
162,7823364,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/7823364
163,7856197,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/7856197
164,7863149,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/7863149
161,7525682,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/7525682
165,8026290,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/8026290
160,1468291,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/1468291
167,8352625,71,1.000000,https://pubmed.ncbi.nlm.nih.gov/8352625
